<a href="https://colab.research.google.com/github/OdetteRoest/ANTMN/blob/master/Thesis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install memory-profiler
%load_ext memory_profiler

In [ ]:
%%memit

import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import random
import os
import networkx as nx
import matplotlib.pyplot as plt
from keras import layers
from keras import Input
import torch
import tensorflow as tf
import torch.nn as nn
import torch.optim as optim
from keras.optimizers import Adam
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader


os.mkdir('students_folder')
os.mkdir('schools_folder')
os.mkdir('MM_TTC_folder')
os.mkdir('MM_PTC')

In [ ]:
%%memit
### Creating data ###

# standard values
number_of_schools = 10
number_of_students = 100
capacity= [10,10,10,10,10,10,10,10,10,10]
number_of_files = 100

# Make array with all student numbers
numbers = []
for i in range(1, number_of_students+1):
    numbers.append(str(i))

# Make student preference lists
def student_lists(sd):
    random.seed(sd)
    schools = ['a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j']
    all_lists = pd.DataFrame(columns=('a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j'))

    for i in range(number_of_students):
        preference_list = random.sample(schools,len(schools))
        all_lists.loc[i] = preference_list

    return all_lists

# Make schools priority lists.
def school_lists(sd,numbers):
    random.seed(sd)
    students = numbers
    all_schoollists = pd.DataFrame(columns=students)

    for i in range(number_of_schools):
        preference_schoollist = random.sample(students,len(students))
        all_schoollists.loc[i] = preference_schoollist

    return all_schoollists

# Change the letter (of a school) to a number
def letter_to_alphabetical_number(letter):
        return ord(letter) - ord('a')

# Create and save the files
for file in range(number_of_files):
    seed = file
    students = student_lists(seed)
    schools = school_lists(seed,numbers)
    students.to_csv(os.path.join("students_folder",  f"students_{file}.csv"))
    schools.to_csv(os.path.join("schools_folder",  f"schools_{file}.csv"))




In [ ]:
%%memit
#creating batches
class BatchDataClass(Dataset):
  def __init__(self, folder_path):
    self.folder_path = folder_path
    self.name_list = os.listdir(folder_path)

  def __len__(self):
    return len(self.name_list)

  def __getitem__(self, index):
    file_name = self.name_list[index]
    number_part = file_name.split('_')[1]
    iteration = int(number_part.split('.')[0])
    schools = os.path.join(folder_path_schools, sorted_file_list_schools[iteration])
    schools = pd.read_csv(schools)
    schools = schools.drop(columns='Unnamed: 0')
    schools_array = schools.values
    schools_array = schools_array.T
    schools = torch.tensor(schools_array, dtype=torch.float32)

    return schools_array, schools, iteration

    # print('index', index)
    # file_name = self.name_list[index]
    # file_path = os.path.join(self.folder_path, file_name)
    # print('file_name',file_name)
    # with open(file_path,'r') as file:
    #   data = file.read()
    # return data, file_name

In [ ]:
%%memit
### Neural network ###

class create_neural_network(nn.Module):
  def __init__(self, input_size, nodes):
    super(create_neural_network, self).__init__()
    self.layer_1 = nn.Linear(input_size, nodes)
    self.layer_2 = nn.Linear(nodes, nodes)
    self.layer_3 = nn.Linear(nodes, nodes)

    # create output layer with number of students as output (weight for each spot)
    self.layer_8 = nn.Linear(nodes, 1)
    #self.output_layer = nn.Sigmoid()

  def forward(self,model):
    model = torch.relu(self.layer_1(model))
    model = torch.relu(self.layer_2(model))
    model = torch.relu(self.layer_3(model))
    model = self.layer_8(model)
    #model = model.view(-1, number_of_schools, number_of_students)
   #model = self.output_layer(model)
    return model


In [ ]:
%%memit
### Decide the pointings of the schools PTC our method###

def find_pointing_of_school(prediction,priority_list, capacity):
  vector_of_pointings= []
  # weight x index of the spot among the other schools.
  for s in range(number_of_schools):
    priority_school = priority_list[:,s]
    non_zero_priority_list = priority_school[priority_school != 0]
    sub_list_scores_school = [] #per school te kijken naar zijn top priorities
    if capacity[s] == 0:
      vector_of_pointings.append(np.nan)
    else:
      for stud in non_zero_priority_list[0:capacity[s]]:
        value_spot = 0
        for compare_school in range(number_of_schools):
          if compare_school == s:
            continue
          else:
            list_comparable = priority_list[:,compare_school]
            list_comparable = list_comparable[list_comparable != 0]
            spot = int(np.where(list_comparable == stud)[0])
            spot = spot + 1 # to get ride of the index value
            value_spot = value_spot + (prediction.at[spot-1,0] * spot) #take the index x the ranking
        sub_list_scores_school.append(value_spot)
      point_index = sub_list_scores_school.index(min(sub_list_scores_school))
      vector_of_pointings.append(int(non_zero_priority_list[point_index]))
  return vector_of_pointings

def find_pointing_of_students(students):
  vector_of_pointings_students = []
  for stud in range(len(students)):
    row = students.iloc[stud]#.to_numpy()
    row_without_zero = row[row != 0]

    if len(row_without_zero) == 0:
      vector_of_pointings_students.append('z')
    else:
      vector_of_pointings_students.append(row_without_zero[0])
  #print(vector_of_pointings_students)

  return vector_of_pointings_students


In [ ]:
%%memit
### Justified envy calculation ###
def justified_envy_score(match_matrix_NN, priority_lists, preferences):
  number_of_situations = 0
  number_of_students_envy = []
  average_envy_on = np.zeros(100)
  number_of_victems = []
  average_victems = np.zeros(100)

  #print('preferences',preferences)
  for student in range(number_of_students):

    top_priority_student = preferences.at[student,'a'] #preference starts at zero!
    #print('top_priority_student', top_priority_student)
    preferences_of_student = (preferences.loc[student].values).tolist()
    #preferences_of_student = preferences.iloc[student].values #(preferences.loc[student].values).tolist()
    #print('preference_of_student',preferences_of_student, type(preferences_of_student))
    #print('tweede', (preferences.loc[student].values).tolist(),type((preferences.loc[student].values).tolist()))
    row_student_MM = match_matrix_NN.loc[student+1] # index starts at 1
    matched_with = np.where(row_student_MM == 1)
    matched_with = number_to_letter(int(matched_with[0]))

    if top_priority_student == matched_with:
      continue
    else:
      #print('matched_with', matched_with)
      #print('preferences_of_student',preferences_of_student)
      schools_before_matched = preferences_of_student[:preferences_of_student.index(matched_with)]
      for better_school in schools_before_matched:
        #print('schools_before_matched',schools_before_matched)
        priority_list_of_better_school = priority_lists[:,letter_to_alphabetical_number(better_school)]
        #print('priority_list_of_better_school', priority_list_of_better_school)
        students_matched_to_better_school_ID = match_matrix_NN[match_matrix_NN[better_school] == 1].index.tolist()
        #print('students_matched_to_better_school_ID',students_matched_to_better_school_ID)
        index_list_student_ID_better_school = [np.where(priority_list_of_better_school == student_id)[0][0] for student_id in students_matched_to_better_school_ID]
        #print('index_list_student_ID_better_school',index_list_student_ID_better_school)
        ID_of_envying_student = np.where(priority_list_of_better_school == student+1)
        #print('ID_of_envying_student', ID_of_envying_student)
        bigger_so_envy = index_list_student_ID_better_school > ID_of_envying_student[0][0]
        #print('bigger_so_envy',bigger_so_envy)

        for indx,comp in enumerate(bigger_so_envy):
          if comp == True: #than the student has envy to the student that is further in the priority list

            #update alles:
            number_of_situations = number_of_situations + 1
            number_of_students_envy.append(student)
            average_envy_on[student] = average_envy_on[student] + 1
            number_of_victems.append(students_matched_to_better_school_ID[indx])
            average_victems[students_matched_to_better_school_ID[indx]-1] = average_victems[students_matched_to_better_school_ID[indx]-1] + 1
  #JE_score = 1/(number_of_situations + 10**(-6))
  JE_score = number_of_situations
  JE_score = torch.tensor(JE_score, dtype=torch.float32,requires_grad = True)
# printing what we want!
  print('number_of_situations', number_of_situations)
  # print('number_of_students_envy', number_of_students_envy)
  # print('average_envy_on', average_envy_on)
  # print('number_of_victems', number_of_victems)
  # print('average_victems', average_victems)
  # print('MEAN average_envy_on', np.mean(average_envy_on), np.mean(average_envy_on[average_envy_on != 0]))
  # print('MEAN average_victems', np.mean(average_victems), np.mean(average_victems[average_victems != 0]))
  # print('amount number_of_students_envy', len(set(number_of_students_envy)))
  # print('amount victems', len(set(number_of_victems)))

  return -JE_score


In [ ]:
%%memit
### Correct input size for the network ###
folder_path_students = '/content/students_folder'
csv_files_students = [file for file in os.listdir(folder_path_students) if file.endswith('.csv')]
sorted_file_list_students = sorted(csv_files_students, key=lambda x: int(x.split('_')[1].split('.')[0]))
folder_path_schools = '/content/schools_folder'
csv_files_schools = [file for file in os.listdir(folder_path_schools) if file.endswith('.csv')]
sorted_file_list_schools = sorted(csv_files_schools, key=lambda x: int(x.split('_')[1].split('.')[0]))

def create_correct_input_size(iteration):
  schools = os.path.join(folder_path_schools, sorted_file_list_schools[iteration])
  schools = pd.read_csv(schools)
  schools = schools.drop(columns='Unnamed: 0')
  schools_array = schools.values
  schools_array = schools_array.T
  schools = torch.tensor(schools_array, dtype=torch.float32)
  input_matrix = schools
  return schools_array, input_matrix

def get_students_files(iteration):
  students = os.path.join(folder_path_students, sorted_file_list_students[iteration])
  students = pd.read_csv(students)
  students = students.drop(columns ='Unnamed: 0')
  #print(type(students))
  return students

In [ ]:
# # prioritized trading cycle

# def find_preference_student_PTC(student):
#   row_PTC = mod_students_PTC.loc[int(student)-1].values
#   row_without_nan_PTC = pd.Series(row_PTC).dropna()
#   if len(row_without_nan_PTC) > 0:
#       pref_s = row_without_nan_PTC.iloc[0]
#       return pref_s
#   else:
#       return None

# #RENEW!
# def find_preference_school_PTC(school):
#   vector_school_indexes = np.zeros((10,), dtype=int)
#   number = letter_to_alphabetical_number(school)
#   total_row_school = mod_schools_PTC.loc[number].values
#   row_school_without_nan_PTC = pd.Series(total_row_school).dropna()
#   mini_list_school = row_school_without_nan_PTC.iloc[0:capacity_PTC[number]]
#   mini_list_school.reset_index(drop=True, inplace=True)
#   #print('capacity ptc number',capacity_PTC[number])
#   if capacity_PTC[number] == 1 or len(available_schools) == 1:
#     pref_c = mini_list_school[0]
#     return int(pref_c)
#   else:
#     for school_av in available_schools:
#       #print('available_schools', available_schools)
#       if school_av != school:
#         row_otherschool = mod_schools_PTC.loc[letter_to_alphabetical_number(school_av)].values
#         row_otherschool_without_NA = pd.Series(row_otherschool).dropna()
#         #print(mini_list_school)
#         for idx, stud in enumerate(mini_list_school):
#           vector_school_indexes[idx] = vector_school_indexes[idx] + list(row_otherschool_without_NA).index(stud)
#     #hao = highest among others
#     # print('taking the min over: ', (vector_school_indexes/len(available_schools)))
#     # print('taking the min over without zeros: ', [np.nonzero(vector_school_indexes/len(available_schools))])
#     hao = np.min((vector_school_indexes/len(available_schools))[np.nonzero(vector_school_indexes/len(available_schools))])

#     index_student = list(vector_school_indexes/len(available_schools)).index(hao)
#     pref_c = int(mini_list_school[index_student])
#     return pref_c

# def letter_to_alphabetical_number(letter):
#   return ord(letter) - ord('a')

# def number_to_letter(number):
#   return chr(number + ord('a'))

# count = -1
# JE_score_PTC = []
# for students, schools in zip(sorted_file_list_students,sorted_file_list_schools):
#   count = count+1
#   print(students)
#   folder_path_students = "/content/students_folder/"
#   students = pd.read_csv(os.path.join(folder_path_students, students))
#   students = students.drop(columns='Unnamed: 0')
#   folder_path_schools = "/content/schools_folder/"
#   schools = pd.read_csv(os.path.join(folder_path_schools, schools))
#   schools = schools.drop(columns='Unnamed: 0')

#   match_matrix_PTC = np.zeros((number_of_students, number_of_schools), dtype=int)
#   unmatched_student_PTC = numbers.copy()
#   available_schools = ['a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j']
#   all_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']#, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

#   mod_students_PTC = students.copy()
#   mod_schools_PTC = schools.copy()
#   capacity_PTC = capacity.copy()

#   while len(unmatched_student_PTC) > 0:
#     # print(unmatched_student_PTC)
#     # print('capacity', capacity_PTC)
#     G = nx.DiGraph()
#     G.add_nodes_from(all_names)
#     #add edges to the graph
#     for s in unmatched_student_PTC:
#       pref_s = find_preference_student_PTC(s)
#       G.add_edge(s, pref_s)

#       # print('s', s, type(s))
#       # print('pref_s', pref_s, type(pref_s))
#     #plt.show()
#     for c in available_schools:
#       pref_c = find_preference_school_PTC(c)
#       # print('c',c,type(c))
#       # print('pref_c', pref_c, type(pref_c))
#       G.add_edge(c, str(pref_c))

#     pos = nx.spring_layout(G)  # Position nodes using the spring layout algorithm
#     nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', arrows=True)
#     plt.close()
#     # Show the graph
#     #plt.show()

#     cycles_PTC = list(nx.simple_cycles(G))
#     #print("Cycles:", cycles_PTC)

#     for cycle in cycles_PTC:
#       start_index = cycle[0].isdigit()
#       #print('cycle0 ', cycle[0], type(cycle[0]))
#       if start_index == False:
#         # Reorder the cycle to start with the number and move one step
#         reordered_cycle = cycle[start_index+1:] + cycle[:start_index+1]

#         # Process the reordered cycle
#         for node in range(0, len(reordered_cycle), 2):
#           first_item = reordered_cycle[node]
#           second_item = reordered_cycle[node + 1]
#           changed_letter = int(chr(ord(second_item) - ord('a') + ord('0')))
#           capacity_PTC[changed_letter] = capacity_PTC[changed_letter] -1
#           match_matrix_PTC[int(first_item)-1, changed_letter] = 1

#           #update
#           #print('first_item', first_item, type(first_item))
#           unmatched_student_PTC.remove(first_item)
#           mod_schools_PTC = mod_schools_PTC.replace(int(first_item), np.nan)
#           if capacity_PTC[changed_letter] == 0:
#             mod_students_PTC = mod_students_PTC.replace(second_item,np.nan)
#             #mod_students = mod_students.apply(pd.to_numeric, errors='coerce').fillna(np.nan).astype(int)
#             available_schools.remove(second_item)
#       else:
#         for node in range(0,len(cycle),2):
#           first_item = cycle[node]
#           second_item = cycle[node + 1]
#           changed_letter = int(chr(ord(second_item) - ord('a') + ord('0')))
#           capacity_PTC[changed_letter] = capacity_PTC[changed_letter] -1
#           match_matrix_PTC[int(first_item)-1, changed_letter] = 1
#           #update
#           unmatched_student_PTC.remove(first_item)
#           mod_schools_PTC = mod_schools_PTC.replace(int(first_item), np.nan)
#           # mod_schools = mod_schools.apply(pd.to_numeric, errors='coerce').fillna(np.nan).astype(int)
#           if capacity_PTC[changed_letter] == 0:
#             mod_students_PTC = mod_students_PTC.replace(second_item,np.nan)
#             #mod_students = mod_students.apply(pd.to_numeric, errors='coerce').fillna(np.nan).astype(int)
#             available_schools.remove(second_item)
# #update the schools and preference lists

#   #transform matrix to data frame

#   match_matrix_PTC = pd.DataFrame(match_matrix_PTC, index=list(range(1,number_of_students+1)), columns=('a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j'))
#   schools_JE = schools.values
#   schools_JE = schools_JE.T
#   JE_PTC = justified_envy_score(match_matrix_PTC, schools_JE, students)
#   JE_score_PTC.append(JE_PTC)
#   #print(JE_score_PTC)
#   #print('match_matrix', match_matrix_PTC)
#   match_matrix_PTC.to_csv(os.path.join("MM_PTC",  f"MM_PTC_{count}.csv"))

# JE_score_PTC = torch.stack(JE_score_PTC)
# mean_PTC = torch.mean(JE_score_PTC).item()
# print(mean_PTC)

# #find the fairnes measures!

In [ ]:
%%memit
def letter_to_alphabetical_number(letter):
  return ord(letter) - ord('a')

def number_to_letter(number):
  return chr(number + ord('a'))

def TTC(vector_schools, vector_students, match_matrix_NN, priority_lists, students_preferences, capacity):
  # In a round of TTC new mathces will be made!
  Graph = nx.DiGraph()
  Graph.add_nodes_from(all_names)
  # add pointings of the students
  for s in range(len(vector_students)):
    if vector_students == 'z': #waar veranderen we de school naar 'z'?
      continue
    else:

      Graph.add_edge(s+1,vector_students[s])

  # add pointings of the schools
  for c in range(len(vector_schools)): # EVEN GOED BEDENKEN WAT ER GEBEURT ALS EEN SCHOOL NAAR NUL WIJST
    if np.isnan(vector_schools[c]):
      continue
    else:
      Graph.add_edge(number_to_letter(c), vector_schools[c])

  pos = nx.spring_layout(Graph)  # Position nodes using the spring layout algorithm
  nx.draw(Graph, pos, with_labels=True, node_color='lightblue', edge_color='gray', arrows=True)
  plt.close()
  cycles = list(nx.simple_cycles(Graph))

  for cycle in cycles:
    start_index = isinstance(cycle[0], int)
    if start_index == False:
      reordered_cycle = cycle[start_index + 1:] + cycle[:start_index+ 1]
      for node in range(0,len(reordered_cycle),2):
        first_item = reordered_cycle[node]
        second_item = reordered_cycle[node + 1]

        changed_letter = int(letter_to_alphabetical_number(second_item))
        capacity[changed_letter] = capacity[changed_letter] - 1
        match_matrix_NN[int(first_item)-1, changed_letter] = 1

        # updating the lists of the school and the students
        priority_lists = np.where(priority_lists == first_item, int(0),priority_lists)
        students_preferences.loc[first_item -1 ] = 0

        if capacity[changed_letter] == 0:
          students_preferences = students_preferences.applymap(lambda x: 0 if x == second_item else x)

    else:  # the cycle starts with a student!
      for node in range(0,len(cycle),2):
        first_item = cycle[node]
        second_item = cycle[node + 1]
        changed_letter = int(letter_to_alphabetical_number(second_item))
        capacity[changed_letter] = capacity[changed_letter] - 1
        match_matrix_NN[int(first_item) - 1, changed_letter] = 1

        priority_lists = np.where(priority_lists == first_item, int(0),priority_lists)
        students_preferences.loc[first_item -1 ] = 0

        if capacity[changed_letter] == 0:
          students_preferences = students_preferences.applymap(lambda x: 0 if x == second_item else x)

          # students_preferences.replace(second_item, int(0))

  #plt.show()
  # match_matrix_NN = pd.DataFrame(match_matrix_NN, index=list(range(1,number_of_students+1)), columns=('a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j'))
  Graph.clear()
  return match_matrix_NN, priority_lists, students_preferences, capacity

In [ ]:
%%memit
def find_JE_score(prediction,unmatched_students, capacity_NN, students, schools, match_matrix_NN,schools_for_JE,iter):

  while len(unmatched_students) > 0:

    #TTC time!
    vector_of_pointings = find_pointing_of_school(prediction, schools,capacity_NN)
    pref_students = find_pointing_of_students(students)
    #print('pref_students',pref_students)
    match_matrix_NN, schools, students, capacity_NN = TTC(vector_of_pointings,pref_students,match_matrix_NN, schools, students, capacity_NN)

    # update unmatched students! (we kijken waar er dus geen 1 in de rij zit, en dus nog gematched moet worden!)
    unmatched_students = np.where(np.sum(match_matrix_NN, axis=1) == 0)[0]
    #print('unmatched_students',unmatched_students)

  match_matrix_NN = pd.DataFrame(match_matrix_NN, index=list(range(1,number_of_students+1)), columns=('a', 'b', 'c', 'd', 'e','f', 'g', 'h', 'i', 'j') )
  JE_score = justified_envy_score(match_matrix_NN, schools_for_JE , get_students_files(iter))
  mean_PTC = -123.51
  advantage = JE_score - (mean_PTC)
  return advantage

In [ ]:
%%memit
### Training the model ###
num_epoch = 3
num_batches = 8

input_size = 10
nodes = 50
neural_network = create_neural_network(input_size, nodes)

optimizer = optim.Adam(neural_network.parameters(), lr=0.001)
all_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
learning_rate = 0.00001

data_set = BatchDataClass('/content/schools_folder')

for epoch in range(5): #change to num_epoch
  #print(torch.cuda.is_available())
  print('epoch', epoch)
  progress_bar = tqdm(total=10, desc=f'Epoch [{epoch + 1}/{5}]')

  #create the batches!
  dataloader = DataLoader(dataset = data_set, batch_size = num_batches, shuffle = True)

  for schools_JE, b, file_names in dataloader:
    b = b.to('cuda')
    neural_network = neural_network.to('cuda')
    predictions_NN = neural_network(b)
    #print(predictions_NN[0])
    # print(file_names)
    advantage_vector = []
    gradients_vector = None
    list_batch = []
    for r in range(len(b)):
      # calculate JE
      iter = file_names[r]
      schools = schools_JE[r]
      students = get_students_files(iter)
      schools_for_JE = schools_JE[r] #is dit slim en kunnen we niet beter ergens .copy() gebruiken?
      capacity_NN = capacity.copy()
      match_matrix_NN = np.zeros((number_of_students, number_of_schools), dtype=int)
      unmatched_students = np.where(np.sum(match_matrix_NN, axis=1) == 0)[0]
      advantage = find_JE_score(pd.DataFrame(predictions_NN[r].detach().cpu().numpy()),unmatched_students, capacity_NN, students, schools, match_matrix_NN, schools_for_JE,iter) #schools is same as schools_for_JE so we can also copy it.
      #print('advantage', advantage)
      advantage_vector.append(advantage)

    #print(advantage_vector)
    advantage_vector = torch.stack(advantage_vector)
    mean_advantage = torch.mean(advantage_vector)
    gradients = torch.autograd.grad(predictions_NN, neural_network.parameters(), torch.ones_like(predictions_NN))
    #print('len(gradients)',len(gradients))
    mean_grad = tuple(x / 8 for x in gradients)
    #updating the parameters:
    i = 0
    for name, param in neural_network.state_dict().items():
      param = param.to('cuda')
      param.data -= learning_rate * mean_advantage * mean_grad[i]
      i += 1
    del b
    del predictions_NN
    del advantage_vector
    del gradients

